In [ ]:
!pip install -qq medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import io
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

In [ ]:
# DATA
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)

# Fix the issue based on the error message
if "INPUT_SHAPE" not in globals():
    # Define the variable with the correct value
    INPUT_SHAPE = (224, 224, 3)
elif not isinstance(INPUT_SHAPE, tuple):
    # Convert the variable to a tuple
    INPUT_SHAPE = tuple(INPUT_SHAPE)
elif len(INPUT_SHAPE) != 3:
    # Raise an error if the variable is invalid
    raise ValueError("Variable 'INPUT_SHAPE' must have 3 elements.")

NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:
import subprocess
import os

def clone_repo(repo_url):

    # Define the target directory
    target_dir = os.path.basename(repo_url)

    # Construct the git clone command
    cmd = ["git", "clone", repo_url]

    try:
        # Execute the git clone command
        subprocess.run(cmd, check=True)
        print(f"Repository cloned successfully into {target_dir}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while cloning the repository: {e}")

# Example usage
repo_url = "https://github.com/seymanurakti/fight-detection-surv-dataset"

clone_repo(repo_url)


An error occurred while cloning the repository: Command '['git', 'clone', 'https://github.com/seymanurakti/fight-detection-surv-dataset']' returned non-zero exit status 128.


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def load_videos_from_folder(folder, label):
    videos = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            path = os.path.join(folder, filename)
            cap = cv2.VideoCapture(path)
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            videos.append(np.array(frames))
            labels.append(label)
            cap.release()
    return videos, labels

def download_and_prepare_dataset(base_path):
    fight_videos, fight_labels = load_videos_from_folder(os.path.join(base_path, 'fight'), 1)
    no_fight_videos, no_fight_labels = load_videos_from_folder(os.path.join(base_path, 'noFight'), 0)

    videos = fight_videos + no_fight_videos
    labels = fight_labels + no_fight_labels

    # Convert to numpy arrays
    videos = np.array(videos)
    labels = np.array(labels)

    # Split the dataset
    train_videos, test_videos, train_labels, test_labels = train_test_split(videos, labels, test_size=0.2, random_state=42)
    train_videos, valid_videos, train_labels, valid_labels = train_test_split(train_videos, train_labels, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )

# Base path of your dataset
base_path = 'fight-detection-surv-dataset'

# Get the dataset
prepared_dataset = download_and_prepare_dataset(base_path)
(train_videos, train_labels) = prepared_dataset[0]
(valid_videos, valid_labels) = prepared_dataset[1]
(test_videos, test_labels) = prepared_dataset[2]


<ipython-input-7-834da80e1841>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  videos = np.array(videos)


In [ ]:
import numpy as np
import tensorflow as tf

@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


def preprocess_videos(videos, desired_frames, height, width, channels):
    processed_videos = []
    for video in videos:
        # Resize frames and adjust channels
        resized_frames = []
        for frame in video:
            resized_frame = cv2.resize(frame, (width, height))
            if resized_frame.shape[-1] != channels:
                resized_frame = np.expand_dims(resized_frame, axis=-1)  # Add a channel dimension if needed
            resized_frames.append(resized_frame)

        # Now all frames have the same size, proceed with trimming/padding
        resized_video = np.array(resized_frames)
        if resized_video.shape[0] > desired_frames:
            trimmed_video = resized_video[:desired_frames]
        else:
            padding = np.zeros((desired_frames - resized_video.shape[0], height, width, channels))
            trimmed_video = np.concatenate([resized_video, padding], axis=0)

        processed_videos.append(trimmed_video)

    # Convert list of arrays to a single 4D numpy array
    videos_array = np.stack(processed_videos, axis=0)

    return videos_array

DESIRED_FRAMES = 25
HEIGHT = 80
WIDTH = 50
CHANNELS = 3

# Preprocess the videos
train_videos = preprocess_videos(train_videos, DESIRED_FRAMES, HEIGHT, WIDTH, CHANNELS)
valid_videos = preprocess_videos(valid_videos, DESIRED_FRAMES, HEIGHT, WIDTH, CHANNELS)
test_videos = preprocess_videos(test_videos, DESIRED_FRAMES, HEIGHT, WIDTH, CHANNELS)

trainloader = prepare_dataloader(train_videos, train_labels, "train")
validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
testloader = prepare_dataloader(test_videos, test_labels, "test")

In [ ]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [ ]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [ ]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=(25, 80, 50, 3),
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=2,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/60
6/6 [==============================] - 28s 326ms/step - loss: 2.1876 - accuracy: 0.5111 - top-5-accuracy: 1.0000 - val_loss: 1.6282 - val_accuracy: 0.5333 - val_top-5-accuracy: 1.0000
Epoch 2/60
6/6 [==============================] - 0s 56ms/step - loss: 1.1862 - accuracy: 0.4444 - top-5-accuracy: 1.0000 - val_loss: 0.9430 - val_accuracy: 0.4500 - val_top-5-accuracy: 1.0000
Epoch 3/60
6/6 [==============================] - 0s 56ms/step - loss: 0.9353 - accuracy: 0.5222 - top-5-accuracy: 1.0000 - val_loss: 0.7893 - val_accuracy: 0.4667 - val_top-5-accuracy: 1.0000
Epoch 4/60
6/6 [==============================] - 0s 56ms/step - loss: 0.7048 - accuracy: 0.5333 - top-5-accuracy: 1.0000 - val_loss: 0.7722 - val_accuracy: 0.5333 - val_top-5-accuracy: 1.0000
Epoch 5/60
6/6 [==============================] - 0s 56ms/step - loss: 0.7384 - accuracy: 0.4944 - top-5-accuracy: 1.0000 - val_loss: 0.7155 - val_accuracy: 0.4500 - val_top-5-accuracy: 1.0000
Epoch 6/60
6/6 [=================

In [ ]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


model = run_experiment()

Epoch 1/100
6/6 [==============================] - 24s 296ms/step - loss: 2.2587 - accuracy: 0.4944 - top-5-accuracy: 1.0000 - val_loss: 1.6481 - val_accuracy: 0.5333 - val_top-5-accuracy: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 57ms/step - loss: 1.6356 - accuracy: 0.4944 - top-5-accuracy: 1.0000 - val_loss: 0.7326 - val_accuracy: 0.5333 - val_top-5-accuracy: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 57ms/step - loss: 0.8520 - accuracy: 0.5000 - top-5-accuracy: 1.0000 - val_loss: 1.0575 - val_accuracy: 0.4500 - val_top-5-accuracy: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 57ms/step - loss: 0.8303 - accuracy: 0.5167 - top-5-accuracy: 1.0000 - val_loss: 0.7262 - val_accuracy: 0.4833 - val_top-5-accuracy: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 56ms/step - loss: 0.7043 - accuracy: 0.5333 - top-5-accuracy: 1.0000 - val_loss: 0.7802 - val_accuracy: 0.5333 - val_top-5-accuracy: 1.0000
Epoch 6/100
6/6 [===========